# Übung - Metadaten gruppieren & auswerten 

Nachdem wir im vorherigen Abschnitt gelernt haben, wie sich offene Datensätze mit Hilfe von SPARQL gezielt durchsuchen und filtern lassen, möchten wir Dr. Amir Weber in dieser Phase seiner Untersuchung weiter unterstützen. 

Um besser einschätzen zu können, welche Datenquellen für seine Analyse relevant sind, genügt es jedoch nicht, nur einzelne Datensätze zu finden. Es interessiert ihn auch, welche Institutionen regelmäßig entsprechende Daten bereitstellen und in welchen Formaten diese verfügbar sind. Dies kann Aufschluss darüber geben, wie verlässlich und zugänglich bestimmte Datenquellen sind.

In dieser Übung werden wir daher SPARQL nutzen, um Metadaten zu gruppieren und quantitativ auszuwerten. Wir analysieren, in welchen Datenformaten diese Informationen angeboten werden, um Dr. Weber bei der technischen Einschätzung der Weiterverwendbarkeit zu unterstützen.

Diese aggregierten Informationen geben ihm wertvolle Hinweise auf die Datenqualität, Veröffentlichungsfrequenz und technische Nutzbarkeit – und legen damit die Grundlage für fundierte Entscheidungen im weiteren Verlauf seiner Analyse.

## Daten gruppieren

Im ersten Schritt lassen wir uns anzeigen, welche Datenbereitsteller die meisten Datensätze mit dem Begriff „Baumkataster“ im Titel veröffentlicht haben. Durch diese Gruppierung erhält Dr. Weber einen schnellen Überblick darüber, welche Institutionen oder Verwaltungseinheiten in den letzten Jahren besonders aktiv im Bereich der baumbezogenen offenen Daten waren – ein wichtiger Hinweis auf mögliche Hauptquellen für seine weitere Recherche.

:::{admonition} Erklärung des Codes
:class: hinweis, dropdown

Zu den bekannten Prefixes kommt nun `rdfs` dazu, um das RDF Schema, also allgemeine Begriffsdefinitionen einzuschließen.

Der `COUNT` Befehl in SPARQL wird verwendet, um die Anzahl der Ergebnisse zu zählen, die eine bestimmte Bedingung erfüllen. Der `AS` Befehl folgt dem `COUNT` und dient dazu, das Ergebnis der Zählung einer Variablen zuzuweisen, die dann im Ergebnis verwendet werden kann. `DISTINCT` gibt an, dass nur eindeutige ("distinct") Werte der Variable `?datasetTitle` gezählt werden sollen. `AS ?uniqueDatasetTitleCount` weist das Ergebnis der Zählung der Variable `?uniqueDatasetTitleCount` zu. Diese Variable kann dann im Ergebnis verwendet werden, um die Anzahl der eindeutigen Titel (?datasetTitle) für jede `contributorID` anzuzeigen. 

Die Bedingung `(LANG(?datasetTitle) = "" || LANG(?datasetTitle) = "de")` wird verwendet, um sicherzustellen, dass nur Titel (?datasetTitle) ausgewählt werden, die entweder keine Sprachinformation haben ("") oder explizit als Deutsch ("de") gekennzeichnet sind. Dies ist wichtig, da nicht alle Datenbereitsteller Information über die Sprache des Datensatzes einschließen. Die Suche nach Datensätzen mit deutscher Sprachinformation mag überflüsig wirken, denn das Wort "Baumkataster" ist bereits ein deutsches Wort. Trotzdem wollen wir diese Funktion an dieser Stelle hervorheben, weil wir später davon Gebrauch machen werden.
:::

**Code**

In [ ]:
%endpoint https://data.europa.eu/sparql
PREFIX dcat: <http://www.w3.org/ns/dcat#>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX dcatde: <http://dcat-ap.de/def/dcatde/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX pg: <http://dcat-ap.de/def/politicalGeocoding/>

SELECT ?contributorID (COUNT(DISTINCT ?datasetTitle) AS ?uniqueDatasetTitleCount)
WHERE {
  ?datasetURI a dcat:Dataset;
              dct:title ?datasetTitle;
              dcatde:contributorID ?contributorID;
              dct:modified ?modified.
  FILTER((LANG(?datasetTitle) = "" || LANG(?datasetTitle) = "de") && CONTAINS(LCASE(?datasetTitle), "baumkataster"))
  FILTER(
      CONTAINS(STR(?modified), "2015") ||
      CONTAINS(STR(?modified), "2016") ||
      CONTAINS(STR(?modified), "2017") || 
      CONTAINS(STR(?modified), "2018") || 
      CONTAINS(STR(?modified), "2019") ||
      CONTAINS(STR(?modified), "2020") ||
      CONTAINS(STR(?modified), "2021") ||
      CONTAINS(STR(?modified), "2022") ||
      CONTAINS(STR(?modified), "2023") ||
      CONTAINS(STR(?modified), "2024") ||
  	  CONTAINS(STR(?modified), "2025") 


               )
}
GROUP BY ?contributorID

**Output**

In [ ]:
%endpoint https://data.europa.eu/sparql
PREFIX dcat: <http://www.w3.org/ns/dcat#>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX dcatde: <http://dcat-ap.de/def/dcatde/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX pg: <http://dcat-ap.de/def/politicalGeocoding/>

SELECT ?contributorID (COUNT(DISTINCT ?datasetTitle) AS ?uniqueDatasetTitleCount)
WHERE {
  ?datasetURI a dcat:Dataset;
              dct:title ?datasetTitle;
              dcatde:contributorID ?contributorID;
              dct:modified ?modified.
  FILTER((LANG(?datasetTitle) = "" || LANG(?datasetTitle) = "de") && CONTAINS(LCASE(?datasetTitle), "baumkataster"))
  FILTER(
      CONTAINS(STR(?modified), "2015") ||
      CONTAINS(STR(?modified), "2016") ||
      CONTAINS(STR(?modified), "2017") || 
      CONTAINS(STR(?modified), "2018") || 
      CONTAINS(STR(?modified), "2019") ||
      CONTAINS(STR(?modified), "2020") ||
      CONTAINS(STR(?modified), "2021") ||
      CONTAINS(STR(?modified), "2022") ||
      CONTAINS(STR(?modified), "2023") ||
      CONTAINS(STR(?modified), "2024") ||
  	  CONTAINS(STR(?modified), "2025") 


               )
}
GROUP BY ?contributorID

**Erklärung des Ergebnisses**

Jeweils 8 Datensätze stammen aus Nordrhein-Westfalen und Rheinland-Pfalz, 6 aus Bayern und 4 aus Brandenburg.

Auffällig ist, dass ganz verschiedene Institutionen und Verwaltungsebenen offene Daten bereitstellen. Hinter `http://dcat-ap.de/def/contributors/mobilithek` verbirgt sich die <a href="https://mobilithek.info/" class="external-link" target="_blank">Mobilithek</a> des Bundesverkehrsministeriums und neben Bundesländern finden sich auch kommunale Datenbereitsteller.

## Dateiformate

Im Anschluss interessiert sich Dr. Weber dafür, in welchen Formaten die Baumkataster-Daten vorliegen, da dies direkte Auswirkungen auf deren Weiterverwendbarkeit hat. Während offene Formate wie CSV, GeoJSON oder XML eine einfache Analyse ermöglichen, können proprietäre oder unstrukturierte Formate die Weiterverarbeitung erschweren.

Daher erweitern wir unsere SPARQL-Abfrage, um für jeden Datenbereitsteller zu ermitteln, wie viele unterschiedliche Datenformate er im Zusammenhang mit Baumkataster-Daten anbietet. Diese Auswertung hilft Dr. Weber dabei einzuschätzen, bei welchen Anbietern er mit einem breiten und technisch zugänglichen Angebot rechnen kann – ein entscheidender Aspekt für seine geplante Analyse des Baumbestands.

:::{admonition} Erklärung des Codes
:class: hinweis, dropdown

Statt der Titel, zählen wir nun mit dem Befehl `SELECT ?contributorID (COUNT(DISTINCT ?format) AS ?formatCount)` die verschiedenen Formate, die mit den Datensätzen verbunden sind.

Mit dem `WHERE`-Befehl suchen wir nach Datensätzen (`dcat:Dataset`), die die folgenden Eigenschaften haben:
- `dct:title`: einen Titel.
- `dcatde:contributorID`: eine ID des Datenbereitstellers.
- `dct:modified`: ein Datum der letzten Änderung.
sowie mit `OPTIONAL` nach dem Katalog des Datensatzes, wenn er vorhanden ist.

Dazu filtern wir mit `FILTER`, dass der Titel entweder keine Sprachangabe haben kann oder in Deutsch vorliegen muss, das Wort "baumkataster" enthält (unabhängig von Groß- oder Kleinschreibung) und innerhalb des Zeitraums von 2015 bis 2025 veröffentlicht wurden.

`?datasetURI dcat:distribution ?distribution.` und `?distribution dct:format ?format.` geben zudem an, dass der Datensatz eine Distribution und diese ein bestimmtes Format haben muss.

Mit `GROUP BY ?contributorID` werden die Ergebnisse nach dem Datenbereitsteller sortiert.
:::

**Code**

In [ ]:
%endpoint https://data.europa.eu/sparql
PREFIX dcat: <http://www.w3.org/ns/dcat#>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX dcatde: <http://dcat-ap.de/def/dcatde/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX pg: <http://dcat-ap.de/def/politicalGeocoding/>

SELECT ?contributorID (COUNT(DISTINCT ?format) AS ?formatCount)
WHERE {
  ?datasetURI a dcat:Dataset;
              dct:title ?datasetTitle;
              dcatde:contributorID ?contributorID;
              dct:modified ?modified.
  OPTIONAL { ?datasetURI dcat:catalog ?catalog. }
  FILTER((LANG(?datasetTitle) = "" || LANG(?datasetTitle) = "de") && CONTAINS(LCASE(?datasetTitle), "baumkataster"))
  FILTER(
      CONTAINS(STR(?modified), "2015") ||
      CONTAINS(STR(?modified), "2016") ||
      CONTAINS(STR(?modified), "2017") || 
      CONTAINS(STR(?modified), "2018") || 
      CONTAINS(STR(?modified), "2019") ||
      CONTAINS(STR(?modified), "2020") ||
      CONTAINS(STR(?modified), "2021") ||
      CONTAINS(STR(?modified), "2022") ||
      CONTAINS(STR(?modified), "2023") ||
      CONTAINS(STR(?modified), "2024") ||
      CONTAINS(STR(?modified), "2025")
  )

  ?datasetURI dcat:distribution ?distribution.
  ?distribution dct:format ?format.
}
GROUP BY ?contributorID


**Output**

In [ ]:
%endpoint https://data.europa.eu/sparql
PREFIX dcat: <http://www.w3.org/ns/dcat#>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX dcatde: <http://dcat-ap.de/def/dcatde/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX pg: <http://dcat-ap.de/def/politicalGeocoding/>

SELECT ?contributorID (COUNT(DISTINCT ?format) AS ?formatCount)
WHERE {
  ?datasetURI a dcat:Dataset;
              dct:title ?datasetTitle;
              dcatde:contributorID ?contributorID;
              dct:modified ?modified.
  OPTIONAL { ?datasetURI dcat:catalog ?catalog. }
  FILTER((LANG(?datasetTitle) = "" || LANG(?datasetTitle) = "de") && CONTAINS(LCASE(?datasetTitle), "baumkataster"))
  FILTER(
      CONTAINS(STR(?modified), "2015") ||
      CONTAINS(STR(?modified), "2016") ||
      CONTAINS(STR(?modified), "2017") || 
      CONTAINS(STR(?modified), "2018") || 
      CONTAINS(STR(?modified), "2019") ||
      CONTAINS(STR(?modified), "2020") ||
      CONTAINS(STR(?modified), "2021") ||
      CONTAINS(STR(?modified), "2022") ||
      CONTAINS(STR(?modified), "2023") ||
      CONTAINS(STR(?modified), "2024") ||
      CONTAINS(STR(?modified), "2025")
  )

  ?datasetURI dcat:distribution ?distribution.
  ?distribution dct:format ?format.
}
GROUP BY ?contributorID


**Erklärung des Ergebnisses**

wir erhalten eine einfache Liste mit zwei Spalten: den Datenbereitstellern und der Summe von Formaten der eingestellten Datensätze. Das bedeutet, dass NRW mit seinen eingestellten Dateien auf 29 verschiedene Dateiformate kommt und Bayern auf 21. Das heißt allerdings nicht, dass alle Datensätze in jeweils jedem Format vorhanden sind. Ablesbar ist aus dieser Zusammenstellung lediglich ein gewisses Maß an Flexibilität. 

## Eingrenzung der Abfrage

Nachdem Dr. Weber nun einen Überblick darüber erhalten hat, welche Datenbereitsteller aktiv sind und in welchen Formaten die Daten zur Verfügung stehen, möchte er sich für einen konkreten Datensatz entscheiden, um seine Analyse zu starten. Dabei ist es ihm wichtig, den regionalen Bezug zum Raum Berlin-Brandenburg zu wahren, da seine bisherigen Untersuchungen vor allem auf diese Region fokussiert waren.

Er entscheidet sich daher, gezielt nach Datensätzen zu suchen, die von der Plattform „Open Data Brandenburg“ bereitgestellt werden und das Stichwort „Baumkataster“ im Titel führen. Durch diese gezielte Einschränkung kann Dr. Weber sicherstellen, dass die Daten sowohl thematisch relevant als auch geographisch passend sind – und damit eine geeignete Grundlage für die weitere Analyse des Baumbestands in der Region darstellen.

Um direkt auf die Daten zugreifen zu können, erweitern wir die Abfrage um die Zugriffs-URL (accessURL). Diese verweist auf die konkrete Stelle, an der der Datensatz heruntergeladen oder weiterverarbeitet werden kann – ein praktischer Schritt, der es Dr. Weber ermöglicht, ohne Umwege mit der Analyse zu beginnen.

**Code**

In [ ]:
%endpoint https://www.govdata.de/sparql
PREFIX dcat: <http://www.w3.org/ns/dcat#>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX dcatde: <http://dcat-ap.de/def/dcatde/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX pg: <http://dcat-ap.de/def/politicalGeocoding/>

SELECT ?datasetURI ?datasetTitle ?datasetDescription ?accessURL
WHERE {
  ?datasetURI a dcat:Dataset;
              dct:title ?datasetTitle;
              dct:description ?datasetDescription;
              dcatde:contributorID <http://dcat-ap.de/def/contributors/openDataBrandenburg>;
              dct:modified ?modified;
              dcat:distribution ?distribution.
              
  ?distribution dcat:accessURL ?accessURL.
  
  FILTER((LANG(?datasetTitle) = "" || LANG(?datasetTitle) = "de") && CONTAINS(LCASE(?datasetTitle), "baumkataster"))
  
  FILTER(
    CONTAINS(STR(?modified), "2017") || 
    CONTAINS(STR(?modified), "2018") || 
    CONTAINS(STR(?modified), "2019") || 
    CONTAINS(STR(?modified), "2020") || 
    CONTAINS(STR(?modified), "2021") || 
    CONTAINS(STR(?modified), "2022") || 
    CONTAINS(STR(?modified), "2023") || 
    CONTAINS(STR(?modified), "2024") || 
    CONTAINS(STR(?modified), "2025")
  )
}
ORDER BY ?datasetTitle


**Output**

In [ ]:
%endpoint https://www.govdata.de/sparql
PREFIX dcat: <http://www.w3.org/ns/dcat#>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX dcatde: <http://dcat-ap.de/def/dcatde/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX pg: <http://dcat-ap.de/def/politicalGeocoding/>

SELECT ?datasetURI ?datasetTitle ?datasetDescription ?accessURL
WHERE {
  ?datasetURI a dcat:Dataset;
              dct:title ?datasetTitle;
              dct:description ?datasetDescription;
              dcatde:contributorID <http://dcat-ap.de/def/contributors/openDataBrandenburg>;
              dct:modified ?modified;
              dcat:distribution ?distribution.
              
  ?distribution dcat:accessURL ?accessURL.
  
  FILTER((LANG(?datasetTitle) = "" || LANG(?datasetTitle) = "de") && CONTAINS(LCASE(?datasetTitle), "baumkataster"))
  
  FILTER(
    CONTAINS(STR(?modified), "2017") || 
    CONTAINS(STR(?modified), "2018") || 
    CONTAINS(STR(?modified), "2019") || 
    CONTAINS(STR(?modified), "2020") || 
    CONTAINS(STR(?modified), "2021") || 
    CONTAINS(STR(?modified), "2022") || 
    CONTAINS(STR(?modified), "2023") || 
    CONTAINS(STR(?modified), "2024") || 
    CONTAINS(STR(?modified), "2025")
  )
}
ORDER BY ?datasetTitle


Dr. Weber entscheidet sich schließlich, den Datensatz „Baumkataster Frankfurt (Oder) - (WFS-BAUMKATASTER)” genauer unter die Lupe zu nehmen, da er für seine Analyse relevante Informationen zu den Baumstandorten in der Region liefern könnte. Dieser Datensatz scheint zunächst vielversprechend, da er auf den ersten Blick sowohl geografische als auch baumspezifische Daten zu liefern scheint.

Leider stellt Dr. Weber bei der Überprüfung fest, dass sich der betreffende Datensatz nicht direkt herunterladen lässt oder die bereitgestellte URL nicht wie erwartet funktioniert.

Das zeigt: Auch wenn Metadaten vielversprechende Informationen liefern, ist es in der Praxis oft notwendig, die tatsächliche Verfügbarkeit und Zugänglichkeit der Daten manuell zu prüfen. Dr. Weber wird seine Suche daher fortsetzen müssen – eventuell mit einer breiteren geografischen Eingrenzung oder indem er nach anderen relevanten Begriffen sucht.

Da sich der ausgewählte Datensatz nicht wie gewünscht herunterladen lässt, entscheidet sich Dr. Weber dazu, seine Suche thematisch zu erweitern. Ihm ist bewusst, dass nicht alle relevanten Informationen unter dem Begriff „Baumkataster“ veröffentlicht werden. Ein verwandter Begriff, der insbesondere in städtischen Kontexten häufig auftaucht, ist „Baumpflanzungen“ – ein Thema, das ebenfalls Rückschlüsse auf den aktuellen Baumbestand sowie stadtplanerische Maßnahmen erlaubt.

Dr. Weber formuliert daher eine neue SPARQL-Abfrage, die zusätzlich nach Datensätzen sucht, in deren Titel der Begriff „Baumpflanzungen“ vorkommt. So erhofft er sich, weitere geeignete Datensätze zu identifizieren, die ihm bei der Analyse des Baumwachstums und der Nachpflanzungen im Raum Berlin-Brandenburg nützliche Hinweise liefern können.

**Code**

In [ ]:
%endpoint https://www.govdata.de/sparql
PREFIX dcat: <http://www.w3.org/ns/dcat#>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX dcatde: <http://dcat-ap.de/def/dcatde/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX pg: <http://dcat-ap.de/def/politicalGeocoding/>

SELECT ?datasetURI ?datasetTitle ?datasetDescription ?accessURL
WHERE {
  ?datasetURI a dcat:Dataset;
              dct:title ?datasetTitle;
              dct:description ?datasetDescription;
              dct:modified ?modified;
              dcat:distribution ?distribution.

  ?distribution dcat:accessURL ?accessURL.

  FILTER(
    (LANG(?datasetTitle) = "" || LANG(?datasetTitle) = "de") &&
    (
      CONTAINS(LCASE(?datasetTitle), "baumkataster") || 
      CONTAINS(LCASE(?datasetTitle), "baumpflanzungen")
    )
  )

  FILTER(
    CONTAINS(STR(?modified), "2015") || 
    CONTAINS(STR(?modified), "2016") ||
    CONTAINS(STR(?modified), "2017") || 
    CONTAINS(STR(?modified), "2018") || 
    CONTAINS(STR(?modified), "2019") || 
    CONTAINS(STR(?modified), "2020") || 
    CONTAINS(STR(?modified), "2021") || 
    CONTAINS(STR(?modified), "2022") || 
    CONTAINS(STR(?modified), "2023") || 
    CONTAINS(STR(?modified), "2024") || 
    CONTAINS(STR(?modified), "2025")
  )
}
ORDER BY ?datasetTitle
OFFSET 140

**Output**

In [ ]:
%endpoint https://www.govdata.de/sparql
PREFIX dcat: <http://www.w3.org/ns/dcat#>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX dcatde: <http://dcat-ap.de/def/dcatde/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX pg: <http://dcat-ap.de/def/politicalGeocoding/>

SELECT ?datasetURI ?datasetTitle ?datasetDescription ?accessURL
WHERE {
  ?datasetURI a dcat:Dataset;
              dct:title ?datasetTitle;
              dct:description ?datasetDescription;
              dct:modified ?modified;
              dcat:distribution ?distribution.

  ?distribution dcat:accessURL ?accessURL.

  FILTER(
    (LANG(?datasetTitle) = "" || LANG(?datasetTitle) = "de") &&
    (
      CONTAINS(LCASE(?datasetTitle), "baumkataster") || 
      CONTAINS(LCASE(?datasetTitle), "baumpflanzungen")
    )
  )

  FILTER(
    CONTAINS(STR(?modified), "2015") || 
    CONTAINS(STR(?modified), "2016") ||
    CONTAINS(STR(?modified), "2017") || 
    CONTAINS(STR(?modified), "2018") || 
    CONTAINS(STR(?modified), "2019") || 
    CONTAINS(STR(?modified), "2020") || 
    CONTAINS(STR(?modified), "2021") || 
    CONTAINS(STR(?modified), "2022") || 
    CONTAINS(STR(?modified), "2023") || 
    CONTAINS(STR(?modified), "2024") || 
    CONTAINS(STR(?modified), "2025")
  )
}
ORDER BY ?datasetTitle
OFFSET 140

Beim weiteren Durchstöbern der Datensätze – unterstützt durch den Einsatz von OFFSET zur Paginierung der Ergebnisse – stößt Dr. Weber auf einen Datensatz mit dem Titel „Straßenbaumpflanzungen in Steglitz-Zehlendorf“, der ihm sofort ins Auge fällt. Besonders positiv: Der Datensatz steht im CSV-Format zur Verfügung, was eine einfache Weiterverarbeitung ermöglicht.

Ein Blick in die Beschreibung macht jedoch deutlich, dass es sich dabei nicht um das vollständige Baumkataster handelt, sondern um eine Teilmenge, die lediglich bereits erfolgte Neupflanzungen auflistet. Es wird erklärt, dass Pflanzungen erst nach ihrer Durchführung in das eigentliche Baumkataster übernommen werden. Die enthaltenen Einträge stammen aus verschiedenen Quellen – etwa Spendenaktionen oder bezirklichen Projekten – und umfassen zudem Angaben zur Pflege der Bäume.

Dr. Weber erkennt den Wert dieser Information, stellt aber auch fest, dass sie für eine umfassende Analyse des aktuellen Baumbestands nicht ausreicht. Der Datensatz bietet zwar Einblicke in die Entwicklung und Nachpflanzung, jedoch nicht in die vollständige aktuelle Baumverteilung im Bezirk oder gar in Berlin insgesamt. Damit bleibt der umfassende Zugriff auf ein vollständiges Kataster weiterhin ausständig, und Dr. Weber wird seine Suche weiter ausbauen müssen, um eine geeignete Datengrundlage zu finden.

Da sich viele der bisher gefundenen Datensätze entweder als unvollständig oder schwer zugänglich erwiesen haben, blickt Dr. Weber nun in Richtung Nordrhein-Westfalen (NRW). Bereits bei der vorherigen Auswertung zur Datenverfügbarkeit war ihm aufgefallen, dass NRW eine besonders hohe Anzahl an relevanten Datensätzen bereitstellt – und das in verschiedenen Formaten und mit stabiler Zugriffbarkeit.

Die gute technische Performance und die Vielzahl veröffentlichter Inhalte lassen darauf schließen, dass das Land NRW ein fortschrittliches Open-Data-Angebot betreibt, das möglicherweise besser für eine umfassende Analyse geeignet ist. Aus diesem Grund richtet Dr. Weber nun gezielt seine Suche auf Datensätze, die durch Open Data NRW bereitgestellt werden und thematisch zu seiner Untersuchung rund um Baumkataster und Baumpflanzungen passen.

**Code**

In [ ]:
%endpoint https://www.govdata.de/sparql
PREFIX dcat: <http://www.w3.org/ns/dcat#>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX dcatde: <http://dcat-ap.de/def/dcatde/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX pg: <http://dcat-ap.de/def/politicalGeocoding/>

SELECT ?datasetURI ?datasetTitle ?datasetDescription ?accessURL
WHERE {
  ?datasetURI a dcat:Dataset;
              dct:title ?datasetTitle;
              dct:description ?datasetDescription;
              dcatde:contributorID <http://dcat-ap.de/def/contributors/openNRW>;
              dct:modified ?modified.

  OPTIONAL {
    ?datasetURI dcat:distribution ?distribution.
    ?distribution dcat:accessURL ?accessURL.
  }

  FILTER(
    (LANG(?datasetTitle) = "" || LANG(?datasetTitle) = "de") &&
    (CONTAINS(LCASE(?datasetTitle), "baumkataster")
    )
  )

  FILTER(
    CONTAINS(STR(?modified), "2017") || 
    CONTAINS(STR(?modified), "2018") || 
    CONTAINS(STR(?modified), "2019") || 
    CONTAINS(STR(?modified), "2020") || 
    CONTAINS(STR(?modified), "2021") || 
    CONTAINS(STR(?modified), "2022") || 
    CONTAINS(STR(?modified), "2023") || 
    CONTAINS(STR(?modified), "2024") || 
    CONTAINS(STR(?modified), "2025")
  )
}
ORDER BY ?datasetTitle

**Output**

In [ ]:
%endpoint https://www.govdata.de/sparql
PREFIX dcat: <http://www.w3.org/ns/dcat#>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX dcatde: <http://dcat-ap.de/def/dcatde/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX pg: <http://dcat-ap.de/def/politicalGeocoding/>

SELECT ?datasetURI ?datasetTitle ?datasetDescription ?accessURL
WHERE {
  ?datasetURI a dcat:Dataset;
              dct:title ?datasetTitle;
              dct:description ?datasetDescription;
              dcatde:contributorID <http://dcat-ap.de/def/contributors/openNRW>;
              dct:modified ?modified.

  OPTIONAL {
    ?datasetURI dcat:distribution ?distribution.
    ?distribution dcat:accessURL ?accessURL.
  }

  FILTER(
    (LANG(?datasetTitle) = "" || LANG(?datasetTitle) = "de") &&
    (CONTAINS(LCASE(?datasetTitle), "baumkataster")
    )
  )

  FILTER(
    CONTAINS(STR(?modified), "2017") || 
    CONTAINS(STR(?modified), "2018") || 
    CONTAINS(STR(?modified), "2019") || 
    CONTAINS(STR(?modified), "2020") || 
    CONTAINS(STR(?modified), "2021") || 
    CONTAINS(STR(?modified), "2022") || 
    CONTAINS(STR(?modified), "2023") || 
    CONTAINS(STR(?modified), "2024") || 
    CONTAINS(STR(?modified), "2025")
  )
}
ORDER BY ?datasetTitle

**Fund eines geeigneten Datensatzes**

Nach intensiver Recherche entscheidet sich Dr. Weber schließlich für den Datensatz „Baumkataster Köln 2017“. Dieser bietet eine umfassende Beschreibung des städtischen Baumbestands und enthält nicht nur geografische Informationen, sondern auch detaillierte Angaben zur Baumart, zum Alter, zur Kronenbreite, zur Stammdicke sowie zur Wuchshöhe der Bäume.

Besonders positiv fällt Dr. Weber die gute Zugänglichkeit des Datensatzes auf - die enthaltenen Informationen sind klar strukturiert, verständlich dokumentiert und stehen in einem gängigen Format zur Verfügung. Dadurch lässt sich der Datensatz problemlos in bestehende Analyseumgebungen integrieren – ein wichtiger Vorteil für die praktische Weiterverarbeitung. Auch wenn der Datensatz noch nicht den vollständigen Baumbestand der Stadt Köln abbildet, bietet er bereits jetzt eine solide Datenbasis für eine fundierte Analyse.

Damit verfügt Dr. Weber über eine valide Grundlage, um erste Auswertungen zur Baumverteilung und -struktur in einer Großstadt vorzunehmen und mögliche Modellregionen für seine Untersuchung zu identifizieren.